## Analyse Appliences on SmartMeters in Buildings 
### UKDALE2 Dataset

In [10]:
import nilmtk

start_time = '2023-08-01'
end_time = '2023-09-06'
sample_period = 10


ukdale = nilmtk.DataSet("ukdale2.h5")
ukdale.set_window(start=start_time, end=end_time)


arr = []
for j in range(len(ukdale.buildings)):
    arr2 = []
    for i in range(len(ukdale.buildings[j+1].elec.appliances)):
        arr2.append(ukdale.buildings[1].elec.appliances[i].type["type"])
    arr.append(arr2)

common_apps = set(arr[0]).intersection(arr[1], arr[4]) # Übereinstimmungen aus Building 1, 2 und 5
print(common_apps)

{'baby monitor', 'radio', 'toasted sandwich maker', 'laptop computer', 'solar thermal pumping station', 'light', 'external hard disk', 'dish washer', 'mobile phone charger', 'toaster', 'USB hub', 'hair dryer', 'kitchen aid', 'boiler', 'security alarm'}


In [11]:
buildings = [1,2,5]
# len(ukdale.buildings[1].elec.meters)
metersPerBuilding = [53,19,5,6,25]
single_apps = [[],[],[]]

for i in range(3):
    for j in ukdale.buildings[i+1].elec.meters:
        if len(j.appliances) == 1:
            single_apps[i].append(j.appliances[0].type['type'])
print(set(single_apps[0])) # single apps aus Building 1
print(set(single_apps[1])) # single apps aus Building 2
print(set(single_apps[2])) # single apps aus Building 5

{'radio', 'oven', 'solar thermal pumping station', 'desktop computer', 'mobile phone charger', 'baby monitor', 'clothes iron', 'light', 'breadmaker', 'vacuum cleaner', 'dish washer', 'microwave', 'tablet computer charger', 'computer monitor', 'television', 'coffee maker', 'soldering iron', 'fan', 'audio amplifier', 'audio system', 'hair dryer', 'HTPC', 'printer', 'charger', 'hair straighteners', 'active subwoofer', 'fridge freezer', 'boiler'}
{'computer monitor', 'rice cooker', 'computer', 'active speaker', 'broadband router', 'laptop computer', 'fridge', 'games console', 'modem', 'external hard disk', 'dish washer', 'cooker', 'toaster', 'running machine', 'microwave', 'kettle', 'washing machine'}
{'electric space heater', 'kettle', 'projector', 'laptop computer'}


In [12]:
# common_single_apps = set(single_apps[0]).intersection(single_apps[1],single_apps[2]) # keine Übereinstimmungen
common_single_apps = set(single_apps[0]).intersection(single_apps[1]) # Übereinstimmungen von single apps aus Building 1 und 2
common_single_apps

{'computer monitor', 'dish washer', 'microwave'}

In [13]:
apps = common_single_apps.intersection(common_apps) # Übereinstimmungen von common und single apps aus Building 1 und 2 
apps

{'dish washer'}

### Training Config

In [14]:
from nilmtk.api import API
from nilmtk.disaggregate import Mean
from nilmtk_contrib.disaggregate import DAE,Seq2Point, Seq2Seq, RNN

epochs = 1
batch_size = 32

config = {
    'power': {
        'mains': ['apparent','active'],
        'appliance': ['apparent','active']
    },
    'sample_rate': 10,

    'appliances': ['microwave', 'dish washer', 'washer dryer', 'coffee maker','computer monitor','laptop computer','audio system','light'],
    'methods': {
        'RNN':RNN({'n_epochs':epochs,'batch_size':batch_size}),
        'DAE':DAE({'n_epochs':epochs,'batch_size':batch_size}),
        'Seq2Point':Seq2Point({'n_epochs':epochs,'batch_size':batch_size}),
        'Seq2Seq':Seq2Seq({'n_epochs':epochs,'batch_size':batch_size}),
        'Mean': Mean({}),
    },
    'train': {
        'datasets': {
            'Dataport': {
                'path': 'ukdale2.h5',
                'buildings': {
                    1: {
                        'start_time': '2015-04-04',
                        'end_time': '2015-04-05'
                    },
                    2: {
                        'start_time': '2015-01-28',
                        'end_time': '2015-01-30'
                        },
                }
            }
        }
    },
    'test': {
        'datasets': {
            'Datport': {
                'path': 'ukdale2.h5',
                'buildings': {
                    1: {
                        'start_time': '2015-04-05',
                        'end_time': '2015-04-06'
                    },
                    2: {
                        'start_time': '2015-04-05',
                        'end_time': '2015-04-06'
                    },
                }
            }
        },
        'metrics':['mae', 'rmse']
    }
}

In [54]:
df = next(ukdale.buildings.elec['microwave'].load())

AttributeError: 'collections.OrderedDict' object has no attribute 'elec'

In [58]:

ukdale = nilmtk.DataSet("ukdale2.h5")

# {'baby monitor', 'radio', 'toasted sandwich maker', 'laptop computer', 'solar thermal pumping station', 'light', 'external hard disk', 'dish washer', 'mobile phone charger', 'toaster', 'USB hub', 'hair dryer', 'kitchen aid', 'boiler', 'security alarm'}

apps = ['microwave', 'dish washer', 'washer dryer', 'radio','computer monitor','laptop computer','security alarm','hair dryer']
timerange = []
temp = []

for i in apps:
    temp = [i]
    for j in range(5):
        try:
            df = next(ukdale.buildings[j+1].elec[i].load())
            temp.append((f'building {j+1}',df.index.min(),df.index.max()))
            timerange.append(temp)
        except:
            print(f'{i} not in building {j+1}')




microwave not in building 3
dish washer not in building 3
dish washer not in building 4
washer dryer not in building 3
radio not in building 2
radio not in building 3
radio not in building 5
computer monitor not in building 3
computer monitor not in building 4
laptop computer not in building 4
laptop computer not in building 5
security alarm not in building 2
security alarm not in building 3
security alarm not in building 4
security alarm not in building 5
hair dryer not in building 2
hair dryer not in building 3
hair dryer not in building 4


In [61]:
timerange[1]

['microwave',
 ('building 1',
  Timestamp('2012-12-14 22:21:33+0000', tz='Europe/London'),
  Timestamp('2017-04-26 18:33:03+0100', tz='Europe/London')),
 ('building 2',
  Timestamp('2013-05-20 22:28:39+0100', tz='Europe/London'),
  Timestamp('2013-10-10 06:15:26+0100', tz='Europe/London')),
 ('building 4',
  Timestamp('2013-03-09 14:40:13+0000', tz='Europe/London'),
  Timestamp('2013-10-01 06:15:14+0100', tz='Europe/London')),
 ('building 5',
  Timestamp('2014-06-29 17:23:56+0100', tz='Europe/London'),
  Timestamp('2014-11-13 17:55:14+0000', tz='Europe/London'))]